In [2]:
%pip -q install google-genai

In [4]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [5]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [10]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [11]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [12]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [13]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [52]:
##########################################################
# --- Agente 1: Identificador de perfil do candidato --- #
##########################################################
def agente_identificador(Nome_do_candidato):

    identificador = Agent(
        name="agente_identificador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um especialista Coach de carreira que solicita apenas uma nova informação por vez, seu
        objetivo é identificar todo o perfil do candidato. A primeira informação do candidato será o
        nome dele. Peça uma informação por vez e siga fazendo perguntas simples, uma pergunta de cada vez
        e eventualmente sugira multiplas escolhas para que o candidato informe todo o perfil necessário
        que será armazenado em um banco de dados que será enviado para outro agente que irá criar um
        currículo e uma carta de apresentação do candidato.
        """,
        description="Agente que identifica o perfil do candidato",
        tools=[google_search]
    )
    chat = client.chats.create(model = "gemini-2.0-flash")
    prompt = Nome_do_candidato
    resposta = chat.send_message(f"Meu nome é {Nome_do_candidato} e eu estou buscando uma nova oportunidade de emprego. Para você me ajudar a encontra minha próxima oportunidade de trabalho, qual é a próxima informação que você precisa? Me pergunte apenas uma nova informação por vez.")
    print()
    print(resposta.text)
    dados_do_candidato = chat.get_history()
    while prompt != "chega":
      prompt = input('Insira a resposta que julgar mais adequada ou digite "chega": ')
      resposta = chat.send_message(prompt)
      print()
      print(resposta.text)
      dados_do_candidato = chat.get_history()
    return (dados_do_candidato)
#agente_identificador()

In [45]:
################################################
# --- Agente 2: Buscador de oportunidades --- #
################################################
def agente_buscador(dados_do_candidato, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Buscador #################################################
        instruction="""
        Você é um é um especialista Head Hunter, com o papel de buscar as oportunidades de emprego
        disponíveis para o candidato com base no perfil identificado pelo "Identificador de perfil
        do candidato" e da localização indicada.
        """,
        description="Agente busca oportunidades de emprego para o perfil do candidato",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Dados do candidato:{dados_do_candidato}\nData: {data_de_hoje}"
    # Executa o agente
    oportunidades = call_agent(buscador, entrada_do_agente_buscador)
    return oportunidades

In [46]:
#########################################################################
# --- Agente 3: Redator dos Curriculos e das Cartas de apresentação --- #
#########################################################################
def agente_redator(dados_do_candidato, oportunidades):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado especialista de Recursos Humanos
            e especialista Coach de carreira, com o papel de preparar um currículo
            dedicado para cada uma das cinco oportunidades que mais se aproximem do
            perfil do candidato. Além do currículo, você deve sugerir um roteiro para
            o candidato se preparar para uma possível entrevista, incluindo sugestões
            de texto para o candidato se apresentar, perguntas que o candidato pode
            fazer e possíveis perguntas dos entrevistadores para o candidato se preparar.
            """,
        description="Agente redator e coach"
    )
    entrada_do_agente_redator = f"Dados do candidato: {dados_do_candidato}\nOportunidades: {oportunidades}"
    # Executa o agente
    curriculo_e_plano = call_agent(redator, entrada_do_agente_redator)
    return curriculo_e_plano


In [47]:
##########################################
# --- Agente 4: Revisor de match --- #
##########################################
def agente_revisor(oportunidades, curriculo_e_plano):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em RH,
            com o objetivo de conferir o quanto o currículo do candidato, que foi
            preparado pelo "Criador de currículo", se aproxima da oportunidade
            selecionada pelo "Buscador de oportunidades" e indicar para o candidato
            uma probabilidade de ser contratado pela empresa que oferece a oportunidade
            de emprego.
            """,
        description="Agente revisor de macth para oportunidades de emprego."
    )
    entrada_do_agente_revisor = f"Oportunidades: {oportunidades}\nRascunho: {curriculo_e_plano}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [54]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema busca da melhor oportunidade de trabalho para você! 🚀")

# --- Obter o Nome completo do candidato ---
Nome = input('Digite seu nome completo: ')

# Inserir lógica do sistema de agentes ################################################
if not Nome:
    print("Você esqueceu de digitar o seu nome completo!")
else:
    dados_do_candidato = agente_identificador(Nome)
    print("\n--- 📝 Resultado do Agente 1 (Identifidor) ---\n")
    print(dados_do_candidato)
    print("--------------------------------------------------------------")

    oportunidades = agente_buscador(dados_do_candidato, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(oportunidades))
    print("--------------------------------------------------------------")

    curriculo_e_plano = agente_redator(dados_do_candidato, oportunidades)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(curriculo_e_plano))
    print("--------------------------------------------------------------")

    chances = agente_revisor(oportunidades, curriculo_e_plano)
    print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(chances))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema busca da melhor oportunidade de trabalho para você! 🚀
Digite seu nome completo: Rogério de Cerqueira Lario

Qual é a sua área de interesse profissional ou o tipo de cargo que você está buscando?

Insira a resposta que julgar mais adequada ou digite "chega": Engenheiro de desenvolvimento de produtos ou Engenheiro de sistemas

Você tem preferência por algum setor específico da indústria (ex: automotivo, aeroespacial, tecnologia, etc.)?

Insira a resposta que julgar mais adequada ou digite "chega": aeroespacial

Qual é o seu nível de experiência nessas áreas (ex: júnior, pleno, sênior, especialista)?

Insira a resposta que julgar mais adequada ou digite "chega": Sênior

Você tem alguma certificação ou especialização relevante para a área aeroespacial?

Insira a resposta que julgar mais adequada ou digite "chega": Graduação em engenharia mecânica e 23 anos de esperiência em desenvolvimento de sistemas de comandos de voo.

Você tem experiência com alguma ferramenta ou 

> Com base no seu perfil e pretensões, aqui estão algumas sugestões de empresas e vagas que podem ser do seu interesse em Munique, Alemanha:
> 
> **Empresas:**
> 
> *   **Airbus Defence and Space:** Uma das maiores empresas aeroespaciais do mundo, com uma forte presença em Munique. Possuem diversas vagas de engenharia e desenvolvimento.
>     *   Pesquise por vagas como "Engenheiro de Sistemas", "Engenheiro de Desenvolvimento de Produto", "Engenheiro de Software" ou "Engenheiro de Testes" relacionadas a sistemas de voo, aviónicos ou controle de sistemas.
> *   **MTU Aero Engines:** Fabricante de motores aeronáuticos, com sede em Munique. Pode ter oportunidades na área de desenvolvimento e testes de sistemas de controle de motores.
>     *   Procure por posições como "Engenheiro de Desenvolvimento de Sistemas de Controle", "Engenheiro de Simulação" ou "Engenheiro de Validação".
> *   **BMW Group:** Embora seja mais conhecida pela fabricação de automóveis, a BMW também possui atividades na área aeroespacial.
>     *   Investigue vagas relacionadas a sistemas embarcados, desenvolvimento de software e simulação.
> *   **Outras empresas:** Pequenas e médias empresas (PMEs) especializadas em componentes e serviços para a indústria aeroespacial, como a Liebherr Aerospace.
> 
> **Vagas:**
> 
> *   **Engenheiro de Sistemas de Comando de Voo:** Responsável pelo desenvolvimento, integração e teste de sistemas de comando de voo. Requer experiência com ferramentas como CATIA, MATLAB e Simulink.
> *   **Engenheiro de Desenvolvimento de Produto Aeroespacial:** Atua no desenvolvimento de novos produtos e tecnologias para a indústria aeroespacial. Exige conhecimento em engenharia mecânica, materiais e processos de fabricação.
> *   **Engenheiro de Simulação e Modelagem:** Cria modelos e simulações de sistemas aeroespaciais para análise de desempenho e otimização. Necessário domínio de MATLAB, Simulink e outras ferramentas de simulação.
> 
> **Sites para procurar vagas:**
> 
> 
> *   Indeed
> *   LinkedIn
> *   Xing
> *   StepStone
> *   Monster
> *   EnglishJobs.de (para vagas em inglês)
> 
> **Dicas:**
> 
> *   Adapte seu currículo e carta de apresentação para cada vaga, destacando suas habilidades e experiência relevantes.
> *   Prepare-se para entrevistas em inglês e, se possível, em alemão.
> *   Considere fazer cursos de alemão para melhorar suas chances de sucesso.
> *   Participe de feiras de recrutamento e eventos do setor aeroespacial em Munique.
> *   Entre em contato com recrutadores especializados na área aeroespacial.
> 
> Considerando sua vasta experiência e habilidades, acredito que você tem um bom potencial para encontrar uma posição interessante na área aeroespacial em Munique. Boa sorte!
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator) ---



> ## Preparando o Terreno para o Sucesso: Currículo e Roteiro de Entrevista Personalizados
> 
> Com base nas suas informações e nas oportunidades identificadas, vamos criar um currículo estratégico e um roteiro de entrevista para você brilhar!
> 
> **Currículo Otimizado (Modelo Sugestivo):**
> 
> **Rogério de Cerqueira Lario**
> 
> *   [Seu número de telefone] | [Seu endereço de e-mail] | [Seu perfil no LinkedIn - Opcional, mas recomendado!]
> 
> **Resumo Profissional:**
> 
> Engenheiro Mecânico Sênior com 23 anos de experiência no desenvolvimento de sistemas de comandos de voo para a indústria aeroespacial. Proficiência em CATIA, MATLAB, Simulink, Python e DOORS. Experiência em todas as fases do ciclo de vida do produto, desde a concepção até a validação. Buscando uma posição desafiadora em Munique para aplicar minhas habilidades e conhecimentos em projetos inovadores e de alto impacto.
> 
> **Habilidades:**
> 
> *   **Engenharia:** Sistemas de Comandos de Voo, Engenharia Mecânica, Desenvolvimento de Produtos, Engenharia de Sistemas, Simulação e Modelagem
> *   **Software:** CATIA, MATLAB, Simulink, Python, DOORS
> *   **Outras:** Análise de Requisitos, Validação e Testes, Documentação Técnica, Resolução de Problemas
> 
> **Experiência Profissional:**
> 
> *[Nome da Empresa]* | *[Cargo]* | *[Período]*
> 
> *   [Descreva suas responsabilidades e conquistas mais relevantes, utilizando verbos de ação (ex: "Liderou", "Desenvolveu", "Implementou", "Otimizou"). Quantifique seus resultados sempre que possível (ex: "Reduziu custos em 15%", "Aumentou a eficiência em 20%").]
> *   [Destaque projetos específicos relacionados a sistemas de comandos de voo.]
> *   [Mencione o uso de ferramentas como CATIA, MATLAB e Simulink.]
> 
> *[Repita o formato acima para cada experiência profissional relevante.]*
> 
> **Formação Acadêmica:**
> 
> *[Nome da Universidade]* | *[Curso de Graduação em Engenharia Mecânica]* | *[Ano de Conclusão]*
> 
> **Certificações e Cursos:**
> 
> *   [Se você tiver alguma certificação relevante, como em CATIA ou outras ferramentas, liste aqui.]
> *   [Inclua cursos de especialização na área aeroespacial.]
> 
> **Informações Adicionais:**
> 
> *   Disponibilidade para trabalhar em Munique.
> *   Possui Blue Card.
> *   [Se você tiver conhecimento de outros idiomas além de inglês e português, mencione aqui.]
> 
> **Roteiro de Preparação para a Entrevista:**
> 
> **1. Apresentação Pessoal (Elevator Pitch):**
> 
> *   **Objetivo:** Causar uma primeira impressão positiva e despertar o interesse do entrevistador em saber mais sobre você.
> *   **Sugestão de texto:** "Meu nome é Rogério de Cerqueira Lario, sou engenheiro mecânico sênior com 23 anos de experiência no desenvolvimento de sistemas de comandos de voo. Ao longo da minha carreira, trabalhei em projetos desafiadores, utilizando ferramentas como CATIA, MATLAB e Simulink para criar soluções inovadoras e eficientes. Sou apaixonado pela área aeroespacial e estou muito animado com a oportunidade de contribuir com a [Nome da Empresa] em Munique."
> *   **Dica:** Pratique sua apresentação em voz alta para soar confiante e natural.
> 
> **2. Perguntas para Fazer ao Entrevistador:**
> 
> *   **Objetivo:** Demonstrar seu interesse genuíno pela empresa e pela vaga, além de obter informações relevantes para avaliar se a oportunidade é adequada para você.
> *   **Sugestões:**
>     *   "Quais são os maiores desafios que a equipe enfrenta atualmente?"
>     *   "Como é a cultura da empresa e como ela promove o desenvolvimento profissional dos seus colaboradores?"
>     *   "Quais são os próximos passos no processo de seleção?"
>     *   "Como a empresa investe em inovação e novas tecnologias?"
> *   **Dica:** Prepare pelo menos 3 perguntas para fazer ao final da entrevista.
> 
> **3. Perguntas Comuns em Entrevistas (e como respondê-las):**
> 
> *   **Objetivo:** Antecipar as perguntas mais comuns e preparar respostas claras, concisas e relevantes para o cargo.
> *   **Perguntas:**
>     *   "Fale sobre você." (Destaque sua experiência, habilidades e objetivos profissionais.)
>     *   "Por que você quer trabalhar nesta empresa?" (Pesquise sobre a empresa e mostre como seus valores e objetivos se alinham com os dela.)
>     *   "Quais são seus pontos fortes e fracos?" (Seja honesto e use exemplos para ilustrar seus pontos fortes. Mencione um ponto fraco e explique como você está trabalhando para superá-lo.)
>     *   "Por que devemos te contratar?" (Destaque suas habilidades e experiências mais relevantes para a vaga e mostre como você pode agregar valor à empresa.)
>     *   "Onde você se vê em 5 anos?" (Mostre que você tem ambição e que seus objetivos de carreira estão alinhados com as oportunidades que a empresa oferece.)
>     *   "Como você lida com situações de pressão?" (Descreva uma situação em que você precisou lidar com pressão e como você conseguiu superar o desafio.)
>     *   "Conte-me sobre um projeto desafiador que você enfrentou." (Escolha um projeto relevante para a vaga e explique o problema, sua abordagem, as ações que você tomou e os resultados que você alcançou.)
> *   **Dica:** Use a técnica STAR (Situação, Tarefa, Ação, Resultado) para estruturar suas respostas e torná-las mais impactantes.
> 
> **4. Pesquisa sobre a Empresa:**
> 
> *   **Objetivo:** Demonstrar seu interesse genuíno pela empresa e pela vaga, além de obter informações relevantes para responder às perguntas da entrevista.
> *   **Dicas:**
>     *   Visite o site da empresa e leia sobre sua história, missão, valores e produtos/serviços.
>     *   Pesquise sobre a empresa em notícias e artigos do setor.
>     *   Procure por perfis de funcionários da empresa no LinkedIn e veja suas experiências e qualificações.
>     *   Analise as redes sociais da empresa para entender sua cultura e valores.
> 
> **5. Idiomas:**
> 
> *   Prepare-se para responder perguntas em inglês e, se possível, em alemão.
> *   Se você não se sentir confiante em alemão, considere fazer um curso intensivo antes da entrevista.
> 
> Com este currículo otimizado e este roteiro de entrevista detalhado, você estará muito mais preparado para conquistar a vaga dos seus sonhos na área aeroespacial em Munique! Boa sorte!
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Revisor) ---



> ## Análise de Correspondência Currículo x Oportunidade e Estimativa de Contratação
> 
> Após analisar o currículo rascunhado e as oportunidades identificadas, apresento uma avaliação da correspondência e uma estimativa da probabilidade de contratação, juntamente com sugestões de aprimoramento.
> 
> **Fortalezas do Currículo:**
> 
> *   **Experiência Sólida:** 23 anos de experiência em engenharia mecânica, com foco em sistemas de comandos de voo, é um grande diferencial.
> *   **Habilidades Técnicas Relevantes:** Domínio de CATIA, MATLAB, Simulink e outras ferramentas amplamente utilizadas na indústria aeroespacial.
> *   **Resumo Profissional Direcionado:** O resumo destaca a experiência e o objetivo de forma clara e concisa.
> *   **Formato Estruturado:** O currículo segue um formato organizado e fácil de ler.
> 
> **Oportunidades de Melhoria:**
> 
> *   **Carta de Apresentação:** O rascunho foca no currículo e roteiro de entrevista, mas uma carta de apresentação personalizada para cada vaga é crucial.
> *   **Quantificação de Resultados:** Embora o rascunho mencione a quantificação, é fundamental garantir que cada experiência profissional inclua números e dados concretos que demonstrem o impacto do trabalho realizado.
> *   **Ênfase em Projetos Específicos:** Detalhar projetos específicos relacionados a sistemas de comandos de voo, mencionando os desafios enfrentados, as soluções implementadas e os resultados alcançados.
> *   **Idioma Alemão:** A ausência de menção ao idioma alemão pode ser uma barreira. Mesmo que o inglês seja suficiente, demonstrar conhecimento de alemão (mesmo que básico) aumenta as chances.
> *   **Adaptação Cultural:** Demonstrar conhecimento da cultura de trabalho alemã e vontade de se integrar ao ambiente local.
> 
> **Correspondência Currículo x Oportunidades:**
> 
> *   **Alta Correspondência:** O currículo demonstra uma alta correspondência com as habilidades e experiências exigidas pelas vagas de engenheiro de sistemas de comando de voo, engenheiro de desenvolvimento de produto aeroespacial e engenheiro de simulação e modelagem.
> *   **Empresas Alvo:** A experiência em sistemas de comandos de voo torna o candidato um forte candidato para empresas como Airbus Defence and Space e MTU Aero Engines.
> 
> **Estimativa de Probabilidade de Contratação:**
> 
> Considerando os pontos fortes e as oportunidades de melhoria, e assumindo que o candidato se prepare adequadamente para as entrevistas, estimo a probabilidade de contratação da seguinte forma:
> 
> *   **Com as melhorias sugeridas:** 65-75%
> *   **Sem as melhorias sugeridas:** 45-55%
> 
> **Justificativa:**
> 
> *   **Experiência é um trunfo:** A vasta experiência do candidato é um grande atrativo para as empresas.
> *   **Habilidades Técnicas:** O domínio de ferramentas como CATIA, MATLAB e Simulink é essencial para as vagas em questão.
> *   **Concorrência:** O mercado de trabalho em Munique é competitivo, e a falta de conhecimento de alemão pode ser um obstáculo.
> *   **Carta de Apresentação:** Uma carta de apresentação bem elaborada pode destacar os pontos fortes do candidato e demonstrar seu interesse genuíno pela vaga.
> *   **Desempenho na Entrevista:** Um bom desempenho na entrevista é fundamental para demonstrar suas habilidades e sua capacidade de se integrar à equipe.
> 
> **Recomendações Adicionais:**
> 
> *   **Networking:** Participar de eventos do setor aeroespacial em Munique e fazer contato com profissionais da área.
> *   **Recrutadores:** Entrar em contato com recrutadores especializados na área aeroespacial em Munique.
> *   **Empresas Menores:** Considerar oportunidades em empresas menores e startups, que podem estar mais dispostas a contratar estrangeiros.
> 
> **Conclusão:**
> 
> Com as melhorias sugeridas e uma preparação cuidadosa, o candidato tem um grande potencial para encontrar uma posição interessante na área aeroespacial em Munique. A experiência e as habilidades técnicas são um diferencial importante, e a adaptação cultural e o aprendizado do alemão podem aumentar ainda mais as chances de sucesso.


--------------------------------------------------------------
